# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.Ms = 30 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel30', 'models','t_dGP_train')

Linesearch     25;  Value 2.745285e+04


In [5]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel30');
models = s.models;


In [6]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans BCM
BCM: SMSE 1.88924555, MSLL -1.9250, NLPD 0.9551
Processing energy_data kmeans PoE
PoE: SMSE 5.51609162, MSLL 2.9959, NLPD 5.8760


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [7]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 2.75557926, MSLL -1.0840, NLPD 1.7961


### 1.1.2 Grid q

In [8]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 2.20407429, MSLL 11.3951, NLPD 14.2752 - q   0.20
TERBCM: SMSE 2.17596357, MSLL 4.0807, NLPD 6.9608 - q   0.40
TERBCM: SMSE 2.26485538, MSLL 1.1548, NLPD 4.0349 - q   0.60
TERBCM: SMSE 2.46201559, MSLL -0.2999, NLPD 2.5802 - q   0.80
TERBCM: SMSE 2.75557926, MSLL -1.0840, NLPD 1.7961 - q   1.00
TERBCM: SMSE 3.11395055, MSLL -1.5209, NLPD 1.3592 - q   1.20
TERBCM: SMSE 3.50271455, MSLL -1.7658, NLPD 1.1143 - q   1.40
TERBCM: SMSE 3.89978743, MSLL -1.9006, NLPD 0.9795 - q   1.60
TERBCM: SMSE 4.29719261, MSLL -1.9711, NLPD 0.9090 - q   1.80
TERBCM: SMSE 4.69373033, MSLL -2.0037, NLPD 0.8764 - q   2.00


In [12]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 2.2:0.2:4.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 5.08771210, MSLL -2.0142, NLPD 0.8659 - q   2.20
TERBCM: SMSE 5.47402180, MSLL -2.0118, NLPD 0.8683 - q   2.40
TERBCM: SMSE 5.84498932, MSLL -2.0022, NLPD 0.8779 - q   2.60
TERBCM: SMSE 6.19308130, MSLL -1.9886, NLPD 0.8915 - q   2.80
TERBCM: SMSE 6.51329944, MSLL -1.9727, NLPD 0.9074 - q   3.00
TERBCM: SMSE 6.80398695, MSLL -1.9555, NLPD 0.9246 - q   3.20
TERBCM: SMSE 7.06616255, MSLL -1.9374, NLPD 0.9427 - q   3.40
TERBCM: SMSE 7.30230731, MSLL -1.9182, NLPD 0.9619 - q   3.60
TERBCM: SMSE 7.51535822, MSLL -1.8980, NLPD 0.9821 - q   3.80
TERBCM: SMSE 7.70814656, MSLL -1.8762, NLPD 1.0039 - q   4.00


### 1.1.3 Find single q

In [33]:
criterion = 'TERBCM';

In [34]:
printf('Optimizing q\r\n');
q1 = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=2.2)
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q1) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 2.2000
q1 =  2.2121
TERBCM: SMSE 5.11128466, MSLL -2.0143, NLPD 0.8658


### 1.1.4 Find multi q

In [36]:
criterion = 'TERBCM';

In [39]:
printf('Optimizing q\r\n');
iqs = 2.2 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   2.2079
   2.2053
   2.2027
   2.2050
   2.2039
   2.2029
   2.2056
   2.2020
   2.2038
   2.2019
   2.2096
   2.2063
   2.2011
   2.2034
   2.2082
   2.2069
   2.2051
   2.2036
   2.2042
   2.2033
   2.2009
   2.2028
   2.2058
   2.2018
   2.2042
   2.2070
   2.2040
   2.2092
   2.2097
   2.2092

ld =  0.10000
Start preprocessing...
Finish preprocessing...
qs =

   2.1083
   2.1285
   2.2027
   2.2050
   2.1549
   2.1905
   2.2042
   2.1298
   2.2036
   2.2016
   2.2096
   2.1699
   2.2011
   2.1871
   2.1948
   2.1617
   2.1880
   2.1865
   2.1836
   2.1665
   2.1528
   2.2027
   2.2058
   2.1993
   2.2042
   2.1597
   2.2040
   2.2092
   2.1975
   2.2091

TERBCM: SMSE 5.05050657, MSLL -2.0089, NLPD 0.8712
ld =  1
Start preprocessing...
Finish preprocessing...
qs =

   2.1171
   2.1516
   2.2027
   2.2050
   2.1657
   2.1939
   2.2043
   2.1503
   2.2036
   2.2017
   2.2096
   2.1788
   2.2011
   2.1913
   2.1976
   2.1776
   2.1917
   2.1915
   2.1862
   2.1731

In [41]:
printf('Optimizing q\r\n');
iqs = 0.4 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-4, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   0.40071
   0.40587
   0.40060
   0.40807
   0.40659
   0.40224
   0.40678
   0.40061
   0.40279
   0.40892
   0.40372
   0.40820
   0.40567
   0.40998
   0.40960
   0.40120
   0.40486
   0.40405
   0.40338
   0.40376
   0.40442
   0.40436
   0.40919
   0.40667
   0.40767
   0.40731
   0.40716
   0.40231
   0.40464
   0.40852

ld =  0.10000
Start preprocessing...
Finish preprocessing...
qs =

   0.34780
   0.37143
   0.40057
   0.40807
   0.38610
   0.39729
   0.40579
   0.36565
   0.40267
   0.40884
   0.40368
   0.39361
   0.40563
   0.40290
   0.40354
   0.38388
   0.39686
   0.39609
   0.38956
   0.38680
   0.38293
   0.40431
   0.40919
   0.40546
   0.40763
   0.38644
   0.40716
   0.40230
   0.39892
   0.40839

TERBCM: SMSE 2.18298583, MSLL 4.4962, NLPD 7.3763
ld =  1
Start preprocessing...
Finish preprocessing...
qs =

   0.50926
   0.51011
   0.40064
   0.40809
   0.50037
   0.42661
   0.40971
   0.50773
   0.40318
   0.40915
   0.40396
   0.47298
   0.406

## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [42]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 2.57884549, MSLL -1.8003, NLPD 1.0798


### 1.2.2 Grid q

In [43]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:4.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGRBCM
TEGRBCM: SMSE 2.06285324, MSLL 1.5211, NLPD 4.4012 - q   0.20
TEGRBCM: SMSE 2.10491338, MSLL -0.4319, NLPD 2.4482 - q   0.40
TEGRBCM: SMSE 2.21298680, MSLL -1.2064, NLPD 1.6737 - q   0.60
TEGRBCM: SMSE 2.37497605, MSLL -1.5922, NLPD 1.2879 - q   0.80
TEGRBCM: SMSE 2.57884549, MSLL -1.8003, NLPD 1.0798 - q   1.00
TEGRBCM: SMSE 2.80477543, MSLL -1.9153, NLPD 0.9648 - q   1.20
TEGRBCM: SMSE 3.03352436, MSLL -1.9782, NLPD 0.9019 - q   1.40
TEGRBCM: SMSE 3.25266185, MSLL -2.0109, NLPD 0.8692 - q   1.60
TEGRBCM: SMSE 3.45658701, MSLL -2.0261, NLPD 0.8540 - q   1.80
TEGRBCM: SMSE 3.64324870, MSLL -2.0308, NLPD 0.8493 - q   2.00
TEGRBCM: SMSE 3.81154432, MSLL -2.0296, NLPD 0.8505 - q   2.20
TEGRBCM: SMSE 3.96064615, MSLL -2.0248, NLPD 0.8553 - q   2.40
TEGRBCM: SMSE 4.09039416, MSLL -2.0181, NLPD 0.8620 - q   2.60
TEGRBCM: SMSE 4.20168334, MSLL -2.0100, NLPD 0.8701 - q   2.80
TEGRBCM: SMSE 4.29639701, MSLL -2.0010, NLPD 0.8791 - q   3.00
TEGRBCM: SMSE 4.37

### 1.2.3 Find single q

In [44]:
criterion = 'TEGRBCM';

In [45]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=2.0);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 2.0000


In [46]:
q

q =  2.0041


In [47]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [48]:
criterion = 'TEGRBCM';

In [49]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   2.0086
   2.0009
   2.0099
   2.0019
   2.0049
   2.0024
   2.0064
   2.0082
   2.0051
   2.0043
   2.0035
   2.0004
   2.0085
   2.0065
   2.0050
   2.0078
   2.0088
   2.0050
   2.0029
   2.0052
   2.0030
   2.0045
   2.0068
   2.0043
   2.0084
   2.0056
   2.0067
   2.0056
   2.0092
   2.0076

ld =  0.10000
Start preprocessing...
Finish preprocessing...
qs =

   2.0086
   2.0009
   2.0099
   2.0019
   2.0027
   2.0006
   2.0063
   2.0075
   2.0051
   2.0043
   2.0035
   1.9954
   2.0085
   2.0039
   2.0037
   2.0027
   2.0075
   2.0032
   2.0019
   2.0037
   2.0021
   2.0045
   2.0068
   2.0040
   2.0084
   2.0033
   2.0067
   2.0056
   2.0079
   2.0076

TEGRBCM: SMSE 3.64776284, MSLL -2.0307, NLPD 0.8494
ld =  1
Start preprocessing...
Finish preprocessing...
qs =

   2.0086
   2.0009
   2.0099
   2.0019
   2.0035
   2.0011
   2.0062
   2.0078
   2.0051
   2.0043
   2.0035
   1.9976
   2.0085
   2.0047
   2.0045
   2.0050
   2.0078
   2.0036
   2.0025
   2.005

In [50]:
printf('Optimizing q\r\n');
iqs = 0.2 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   0.20558
   0.20625
   0.20900
   0.20225
   0.20761
   0.20096
   0.20471
   0.20034
   0.20851
   0.20273
   0.20582
   0.20495
   0.20125
   0.20625
   0.20667
   0.20827
   0.20835
   0.20712
   0.20068
   0.20984
   0.20911
   0.20199
   0.20569
   0.20293
   0.20631
   0.20122
   0.20240
   0.20172
   0.20098
   0.20165

ld =  0.10000
Start preprocessing...
Finish preprocessing...
qs =

   0.20558
   0.20625
   0.20899
   0.20219
   0.15507
   0.14672
   0.20036
   0.23281
   0.20776
   0.20192
   0.20560
   0.10655
   0.20243
   0.11708
   0.19244
   0.12262
   0.17485
   0.16423
   0.19186
   0.23425
   0.22530
   0.20221
   0.20569
   0.19810
   0.20644
   0.17321
   0.20240
   0.20176
   0.18348
   0.20110

TEGRBCM: SMSE 2.06797213, MSLL 1.9002, NLPD 4.7803
ld =  1
Start preprocessing...
Finish preprocessing...
qs =

   0.20558
   0.20625
   0.20900
   0.20282
   0.53629
   0.47378
   0.33619
   0.51362
   0.21572
   0.21913
   0.21229
   0.55404
   0.20

## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [51]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 2.03851176, MSLL -0.9711, NLPD 1.9090


### 1.3.2 Grid q

In [52]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:4.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGPoE
TEGPoE: SMSE 2.05278404, MSLL 11.9416, NLPD 14.8217 - q   0.20
TEGPoE: SMSE 2.04911117, MSLL 4.3625, NLPD 7.2426 - q   0.40
TEGPoE: SMSE 2.04541827, MSLL 1.3332, NLPD 4.2133 - q   0.60
TEGPoE: SMSE 2.04203466, MSLL -0.1682, NLPD 2.7119 - q   0.80
TEGPoE: SMSE 2.03851176, MSLL -0.9711, NLPD 1.9090 - q   1.00
TEGPoE: SMSE 2.03510296, MSLL -1.4100, NLPD 1.4701 - q   1.20
TEGPoE: SMSE 2.03236657, MSLL -1.6457, NLPD 1.2344 - q   1.40
TEGPoE: SMSE 2.02989258, MSLL -1.7635, NLPD 1.1166 - q   1.60
TEGPoE: SMSE 2.02814133, MSLL -1.8109, NLPD 1.0692 - q   1.80
TEGPoE: SMSE 2.02897219, MSLL -1.8160, NLPD 1.0641 - q   2.00
TEGPoE: SMSE 2.02952570, MSLL -1.7949, NLPD 1.0852 - q   2.20
TEGPoE: SMSE 2.03092992, MSLL -1.7577, NLPD 1.1224 - q   2.40
TEGPoE: SMSE 2.03370872, MSLL -1.7104, NLPD 1.1697 - q   2.60
TEGPoE: SMSE 2.04435836, MSLL -1.6571, NLPD 1.2230 - q   2.80
TEGPoE: SMSE 2.06806353, MSLL -1.6005, NLPD 1.2796 - q   3.00
TEGPoE: SMSE 2.08790090, MSLL -1.5

### 1.3.3 Find single q

In [53]:
criterion = 'TEGPoE';

In [54]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=2.0);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 2.0000


In [55]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [56]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   2.0010
   2.0087
   2.0003
   2.0022
   2.0075
   2.0093
   2.0072
   2.0023
   2.0099
   2.0005
   2.0095
   2.0001
   2.0028
   2.0003
   2.0037
   2.0095
   2.0030
   2.0098
   2.0021
   2.0024
   2.0046
   2.0085
   2.0032
   2.0018
   2.0091
   2.0021
   2.0017
   2.0055
   2.0028
   2.0019

ld =  0.10000
Start preprocessing...
Finish preprocessing...
qs =

   2.0294
   2.0342
   2.0003
   2.0022
   2.0382
   2.0135
   2.0080
   2.0306
   2.0101
   2.0008
   2.0097
   2.0042
   2.0030
   2.0068
   2.0122
   2.0234
   2.0094
   2.0153
   2.0267
   2.0476
   2.0510
   2.0087
   2.0032
   2.0028
   2.0093
   2.0333
   2.0017
   2.0056
   2.0111
   2.0021

TEGPoE: SMSE 2.02769294, MSLL -1.8209, NLPD 1.0592
ld =  1
Start preprocessing...
Finish preprocessing...
qs =

   2.0485
   2.0608
   2.0003
   2.0022
   2.0551
   2.0168
   2.0084
   2.0527
   2.0102
   2.0008
   2.0097
   2.0231
   2.0031
   2.0117
   2.0164
   2.0445
   2.0154
   2.0210
   2.0388
   2.0562

In [57]:
printf('Optimizing q\r\n');
iqs = 1.8 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   1.8082
   1.8003
   1.8061
   1.8034
   1.8047
   1.8039
   1.8062
   1.8047
   1.8024
   1.8065
   1.8009
   1.8025
   1.8030
   1.8048
   1.8076
   1.8008
   1.8038
   1.8061
   1.8073
   1.8061
   1.8057
   1.8093
   1.8052
   1.8095
   1.8001
   1.8043
   1.8077
   1.8051
   1.8071
   1.8071

ld =  0.10000
Start preprocessing...
Finish preprocessing...
qs =

   1.8507
   1.8340
   1.8061
   1.8035
   1.8521
   1.8106
   1.8079
   1.8479
   1.8027
   1.8070
   1.8012
   1.8122
   1.8035
   1.8153
   1.8226
   1.8220
   1.8150
   1.8156
   1.8492
   1.8693
   1.8750
   1.8095
   1.8052
   1.8122
   1.8004
   1.8483
   1.8077
   1.8052
   1.8158
   1.8075

TEGPoE: SMSE 2.02675613, MSLL -1.8249, NLPD 1.0552
ld =  1
Start preprocessing...
Finish preprocessing...
qs =

   1.8861
   1.8854
   1.8062
   1.8035
   1.8824
   1.8170
   1.8086
   1.8914
   1.8027
   1.8068
   1.8013
   1.8433
   1.8034
   1.8243
   1.8295
   1.8567
   1.8235
   1.8254
   1.8589
   1.8945